<a href="https://colab.research.google.com/github/sus4157d/almabetter/blob/main/ML_bike_sharing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    -Bike sharing demand prediction



##### **Project Type**    - Regression
##### **Contribution**    - Individual


# **Project Summary -**

I have dataset of seaul city in which we have firstly -Rented Bike count in particular hour . Purpose is to train a model to predict the Need of Bike count for any hour in particular day in year. (day and hour are our independent variable ) . The "Rented Bike Sharing Demand" is what we need to predict for any given conditions .
Next- we have dataset columns
'Date','Hour','Temperature(°C)', 'Humidity(%)',
'Wind speed (m/s)', 'Visibility (10m)', 'Dew point temperature(°C)',
'Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)', 'Seasons',
'Holiday', 'Functioning Day'
Which are independent variable for predicting dependent variable.
I have gone through each column and understand the values it have. Did some Data wrangling like cleaning and modifing data. We have 'Date' feature which i have used for creating feature like "Day_month","week_number","day_year","day_week". these are-what date is?,what week number fall on that day? what day of year is that day is? which day of week is this day? these extraction is usefull to determine the demand according to day in week as we have different shedule in different days . I have ploted their relationship with  our dependent variable . that is positive correlation for 'hour' and 'Temp' feature. But for 'humidity','rain','snow' have negative correlation with demand of bike.
 Next 'day in year' is also important to see weather determination for that day like if it will be spring or summer .
We have next columns describing weather condition of day or hour like we have "Temp","visibility","rain","snow", etc which have their relation with bike demand for that day.
I have ploted distibution of 'bike_count ' which is positive skewed . as i need to apply square root transformation to make it normal . we have some values of it zero so we cannt apply log transformation.
Next we have correlation heatmap which is very high for some features like "Temp" and "Dp_temp" , also for "day_month","day_week" with "week_number" so we removed some of them . calculated vif for numeric feature which is less than 10 which is good count. Next we have categorical feature we need to one hot encoding for it now we are ready for spliting data and modeling fit. I have applied LinearRegression form sklearn for modeling and fitting with GridSearch cross validation and Hypertunning with Lasso and Ridge . I Have also used statsmodel.api for modeling which had a better Evaluation Matrix value .

# **GitHub Link -**

https://github.com/sus4157d/almabetter.git

# **Problem Statement**


We need to train a model which will predict demand of bike for a city for a patricular hour for a particular day with given condition of weather given. We need to predict this demand for bike earlier so that we can get ready for situation.

## ***1. Know Data***

### Import Libraries

In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
from numpy import math

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import GridSearchCV
from scipy.stats import zscore
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

import warnings
warnings.filterwarnings('ignore')
from datetime import date
from datetime import datetime

### Dataset Loading

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load Dataset
df=pd.read_csv("/content/drive/MyDrive/SeoulBikeData.csv",encoding = "ISO-8859-1")

### Dataset First View

In [ ]:
# Dataset First Look
df.tail()

### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count
df.shape

### Dataset Information

In [ ]:
# Dataset Info
df.info()

#### Duplicate Values

In [ ]:
# Dataset Duplicate Value Count
len(df[df.duplicated()])

#### Missing Values/Null Values

In [ ]:
# Missing Values/Null Values Count
df.info()

No NULL values present

### What did you know about your dataset?

We have "rent_bike_count" for dv and 12 more column which are idv. We dont have nan value in data set. mostly have numerical(int and float) datatypes but object are also there which need to interperate. we firstly prepare for numerical data than for categorical data will be given.

## ***2. Understanding  Variables***

In [ ]:
# Dataset Columns
df.columns

In [ ]:
# Dataset Describe
df.describe()

### Check Unique Values for each variable.

In [ ]:
# Check Unique Values for each variable.

In [ ]:
df.columns

In [ ]:
df.Date.unique()

In [ ]:
df["Rented Bike Count"].unique()

In [ ]:
df["Hour"].unique()

In [ ]:
df["Temperature(°C)"].unique()

In [ ]:
df['Humidity(%)'].unique()

In [ ]:
df['Wind speed (m/s)'].unique()

In [ ]:
df['Visibility (10m)'].unique()

In [ ]:
df['Dew point temperature(°C)'].unique()

In [ ]:
df['Solar Radiation (MJ/m2)'].unique()

In [ ]:
df['Rainfall(mm)'].unique()

In [ ]:
df['Snowfall (cm)'].unique()

In [ ]:
df['Seasons'].unique()

In [ ]:
df['Holiday'].unique()

In [ ]:
df[ 'Functioning Day'].unique()

## 3. ***Data Wrangling***

### Data Wrangling Code

In [ ]:
df.info()

In [ ]:
# Renaming the columns for better typing
df.rename(columns={"Rented Bike Count":"Rent_bike_count","Temperature(°C)":"Temp","Humidity(%)":"Humidity","Wind speed (m/s)":"Wind_speed","Visibility (10m)":"Visibility","Dew point temperature(°C)":"Dp_temp","Solar Radiation (MJ/m2)":"Solar_rad","Rainfall(mm)":"Rain","Snowfall (cm)":"Snow","Functioning Day":"Funct_day"},inplace=True)

In [ ]:
# converting date to week no.
df['Date']=pd.to_datetime(df['Date'])
df['Week_number'] = df['Date'].dt.isocalendar().week

In [ ]:
# getting the day in mounth and mounth in year and day in year and day in week
df['day_year']=df['Date'].dt.dayofyear
df['day']=df["Date"].dt.day
df['month']=df["Date"].dt.month
df['day_week']=df['Date'].dt.weekday

In [ ]:
df['Week_number']=df['Week_number'].astype(np.int64)

In [ ]:
df["Temp"]=df['Temp'].astype(int)
df['Wind_speed']=df["Wind_speed"].astype(int)
df['Snow']=df["Snow"].astype(int)
df['Rain']=df["Rain"].astype(int)
df['Dp_temp']=df["Dp_temp"].astype(int)
df['Solar_rad']=df["Solar_rad"].astype(int)


In [ ]:
#getting total hour rented per week
df['total_hour']=df.groupby(["Week_number"]).Hour.sum()
plt.figure(figsize=(8,5))
plt.plot(df['total_hour'])
plt.title("Total Rented Hours for week number")
plt.xlabel("week number")
plt.ylabel("total rented hours")

For some week we have less than normal demand for bikes (8,11,46) weeks but some week have (45,50,10,7) more demand of bikes than normal or average day.

In [ ]:
#getting int value
df['total_hour'].fillna(0,inplace=True)
df['total_hour']=df['total_hour'].astype(int)

In [ ]:
# getting differnt temp ranges and rented bike count
df['Temp_r']=df['Temp'].apply(lambda x:0 if x<4.1 else(1 if x<8.1 else( 2 if x< 13 else( 3 if x< 18 else ( 4 if x<24 else (5 if x<29 else(6 if x<35 else 7)))))) )
df.groupby('Temp_r')['Rent_bike_count'].sum()
plt.figure(figsize=(8,5))
plt.plot(df.groupby('Temp_r')['Rent_bike_count'].sum())
plt.title("Total Bike rented in diff temp range")
plt.xlabel("Temp from -17 to 39 degrees")
plt.ylabel("Total Bike Rented")



### What all manipulations have you done and insights you found?

I have divided temp in different range and had total "rented_bike_count' counts which shows firstly decreasing slightly and then increasing trend with temp increase.

## ***4. Data Vizualization, Storytelling & Experimenting with charts : Understand the relationships between variables***

#### Ploting our dependent variabel that is Rented_bike_count

In [ ]:
df.describe()

In [ ]:
plt.figure(figsize=(8,5))
sns.distplot(df['Rent_bike_count'],color='y')

This is positively skewed distribution for 'Rent_bike_count' we need some transformation to make is normally distributed.

In [ ]:
# ploting with Log10 transformation
plt.figure(figsize=(8,5))
sns.distplot(np.sqrt(df['Rent_bike_count']),color='y')

##### 1. Why did you pick the specific chart?

Its the dependent variable . It is important to find out that if this chart is normally distributed. if not normally distributed we apply some transformation to it.

##### 2. What is/are the insight(s) found from the chart?

our dv contains zero value so we cannt apply log or inverse transformation but go with square root transformation.

Below are the some type of method or way to deal above type of problem.

square-root for moderate skew: sqrt(x) for positively skewed data, sqrt(max(x+1) - x) for negatively skewed data

log for greater skew: log10(x) for positively skewed data, log10(max(x+1) - x) for negatively skewed data

inverse for severe skew: 1/x for positively skewed data 1/(max(x+1) - x) for negatively skewed data

#### ploting Numerical Columns of features

In [ ]:
df.head()

In [ ]:
num_f=df.describe().columns

In [ ]:
for col in num_f[1:]:
  fig=plt.figure(figsize=(8,5))
  ax=fig.gca()
  feature=df[col]
  feature.hist(bins=50,ax=ax)
  ax.set_title(col)
  ax.axvline(feature.mean(),color='red',linestyle='dashed',linewidth=2)
  ax.axvline(feature.median(),color='gray',linestyle='dashed',linewidth=2)
  plt.show()

##### 1. Why did you pick the specific chart?

I am ploting all feature variable . to know which type of distibution these varible have .If these are not normally distributed then we can have some transformation applied to these.

##### 2. What is/are the insight(s) found from the chart?

we have some feature like 'Temp','Humidity' have normal distibution but 'windspeed','Solar_rad','Rain',"Snow" etc have positive distribution where as 'visibility'have negative distribution .

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

For better trained model fit and prediction we need to have normally distributed data . Here we need to apply squareroot transformation for postive and for negative sqrt(max(x+1) - x) transformation.

#### Transforming the feature with log transformation

In [ ]:
num_f=num_f.drop("Visibility")

In [ ]:
num_f=num_f.drop("Rent_bike_count")

In [ ]:
for col in num_f[:]:
  fig=plt.figure(figsize=(8,5))
  ax=fig.gca()
  feature=np.sqrt(df[col])
  feature.hist(bins=50,ax=ax)
  ax.set_title(col)
  ax.axvline(feature.mean(),color='red',linestyle='dashed',linewidth=2)
  ax.axvline(feature.median(),color='gray',linestyle='dashed',linewidth=2)
  plt.show()

In [ ]:

  fig=plt.figure(figsize=(8,5))
  ax=fig.gca()
  feature=np.sqrt(max(df["Visibility"]+1) - df["Visibility"])
  feature.hist(bins=50,ax=ax)
  ax.set_title("Visibility")
  ax.axvline(feature.mean(),color='red',linestyle='dashed',linewidth=2)
  ax.axvline(feature.median(),color='gray',linestyle='dashed',linewidth=2)
  plt.show()

For "Visibility" we have used sqrt(max(x+1) - x) transformation where as for "Temp","Dp_temp","Wind_speed","Rain","Snow" etc we have applied squareroot transformation.

#### Drawing Relationship between Dependent Variable and Independent Varaiable

In [ ]:
#scatter plot
num_f=df.describe().columns
num_f=num_f.drop("Rent_bike_count")
num_f

In [ ]:
for col in num_f[:]:
  fig=plt.figure(figsize=(8,5))
  ax=fig.gca()
  feature=df[col]
  label=df['Rent_bike_count']
  correlation=label.corr(feature)
  plt.scatter(x=feature,y=label)
  plt.ylabel("Bike_count")
  plt.xlabel(col)
  plt.title("Bike_count vs " + col + "--correlation "+str(correlation))
  z=np.polyfit(feature,label,1)
  y_hat=np.poly1d(z)(feature)
  plt.plot(feature,y_hat,"r--",lw=1)
plt.show()

##### 1. Why did you pick the specific chart?

Next we need to draw Relationship between Dependent varible and feature variable that is independent varible . we have ploted linear regression line for all feature. which tells the relation ship.

##### 2. What is/are the insight(s) found from the chart?

We have some feature with positive correlation and some feature with negative correlation  for which we need to find coefficent to train the model.

#### Correlation Heatmap

In [ ]:
# Correlation Heatmap
plt.figure(figsize=(18,15))
correlation=df.corr()
sns.heatmap(abs(correlation),annot=True,cmap="coolwarm")

From correlation Heatmap it is clear that "week_number","Temp_r","month","Dp_temp" shows high multicolinearity so we removed these features .
Where as "hour","temp","humidity","wind_speed","day","day_year","day_week","snow","rain" show good relation with "rent_bike_count" which is good for our model.

#### Dealing Multicolinearity

In [ ]:
# Multicolinearity
def calc_vif(X):

   # Calculating VIF
   vif = pd.DataFrame()
   vif["variables"] = X.columns
   vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

   return(vif)

In [ ]:
vif=calc_vif(df[[i for i in df.describe().columns if i not in ['month','Week_number','Rent_bike_count','Temp_r',"Dp_temp"]]])

In [ ]:
vif

### We got vif less then 6.589 which is good for us to carry linear Regression.

In [ ]:
num_f=list(vif['variables'])

In [ ]:
num_f

#### Analysis of Categorical Data

In [ ]:
cat_f=df.describe(include=['object','category']).columns
cat_f

In [ ]:
for col in cat_f:
  counts=df[col].value_counts().sort_index()
  fig=plt.figure(figsize=(8,5))
  ax=fig.gca()
  counts.plot.bar(ax=ax,color='red')
  plt.title(col + " counts ")
  plt.xlabel(col)
  plt.ylabel("freq")
plt.show()

This chart show catogorical feature and there value count on plot .

### boxplot for label for each col

In [ ]:
# ploting box for each col
for col in cat_f:
  fig=plt.figure(figsize=(8,5))
  ax=fig.gca()
  df.boxplot(column="Rent_bike_count",by=col,ax=ax)
  ax.set_title("label by "+col)
  ax.set_title("Bike_count")
plt.show()

we have boxplot for each categorical feature with respect to label . This plot shows that we have outlier in our values of bike count .we need to take care of these.

## Feature Engineering & Data Pre-processing

In [ ]:
# Handling Missing Values & Missing Value Imputation
df.info()

We dont have missing value.

Preprocessing

In [ ]:
df.shape

In [ ]:
dfc=df.copy()

In [ ]:
dfc.shape

# Categorical Encoding

We shall convert categorical features to numeric type...

In [ ]:
# Encode your categorical columns
encoder_num={'Holiday':{'Holiday':0,'No Holiday':1},
             'Funct_day':{'No':0,'Yes':1}}
dfc=dfc.replace(encoder_num)

In [ ]:
#One Hot Encoding
dfc=pd.get_dummies(dfc,columns=["Seasons"],prefix=["Seasons"])

In [ ]:
dfc.head()

##with the help of One Hot Encoding
###Holiday is divided in two attribute.
###Funct_day is divided in two new attribute

In [ ]:
dfc.columns

In [ ]:
feature=num_f

In [ ]:
feature.extend(['Funct_day', 'Holiday','Seasons_Autumn',	'Seasons_Spring',	'Seasons_Summer',	'Seasons_Winter'])

In [ ]:
feature

In [ ]:
#scaling of idv
X=dfc[feature].apply(zscore)

In [ ]:
y=np.sqrt(dfc["Rent_bike_count"])

### 8. Data Splitting

In [ ]:
# Split your data to train and test. Choose Splitting ratio wisely.
X_train, X_test, y_train, y_test = train_test_split( X,y , test_size = 0.2, random_state = 0)
print(X_train.shape)
print(X_test.shape)

##  ML Model Implementation***

Let's now begin to train out regression model! We will need to first split up our data into an train array that contains the part of dataset used for training data, and a test array used for test data.

In [ ]:
# ML Model - 1 Implementation
# Fit the Algorithm
reg = LinearRegression().fit(X_train, y_train)
print(reg.score(X_train, y_train))
print(reg.coef_)

# Predict on the model
y_pred = reg.predict(X_test)

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

Here are three common evaluation metrics for regression problems:

Mean Absolute Error (MAE) is the mean of the absolute value of the errors:

1n∑i=1n|yi−y^i|

Mean Squared Error (MSE) is the mean of the squared errors:

1n∑i=1n(yi−y^i)2

Root Mean Squared Error (RMSE) bold text is the square root of the mean of the squared errors:

1n∑i=√1n(yi−y^i)2

Comparing these metrics:

MAE is the easiest to understand, because it's the average error.
MSE is more popular than MAE, because MSE "punishes" larger errors, which tends to be useful in the real world.
RMSE is even more popular than MSE, because RMSE is interpretable in the "y" units.
All of these are loss functions, because we want to minimize them.

In [ ]:
# Visualizing evaluation Metric Score chart
def eval_metric(y_test,y_pred):
  MSE  = mean_squared_error((y_test)**2, (y_pred)**2)
  print("MSE :" , MSE)

  MAE=mean_absolute_error((y_test)**2, (y_pred)**2)
  print("MAE :" ,MAE)

  RMSE = np.sqrt(MSE)
  print("RMSE :" ,RMSE)

  r2 = r2_score((y_test)**2, (y_pred)**2)
  print("R2 :" ,r2)
  print("Adjusted R2 : ",1-(1-r2_score((y_test)**2, (y_pred)**2))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))

In [ ]:
eval_metric(y_test,y_pred)

In [ ]:
plt.scatter((y_test)**2, (y_pred)**2)
plt.xlabel('Actual')
plt.ylabel('Predicted')

In [ ]:
plt.figure(figsize=(15,10))
plt.plot((y_pred)**2)
plt.plot(np.array((y_test)**2))
plt.legend(["Predicted","Actual"])
plt.xlabel('No of Test Data')
plt.show()

If We see above graph our prediction is quiet good.!:)

#Residuals:

A residual is the vertical distance between a data point and the regression line. Each data point has one residual. They are positive if they are above the regression line and negative if they are below the regression line

In [ ]:
fig=plt.figure(figsize=(8,8))

sns.distplot(((y_test)**2- (y_pred)**2),bins=20)

#Plot Label
fig.suptitle('Residual Analysis', fontsize = 20)

###Cross- Validation & Hyperparameter Tuning

##with Lasso

In [ ]:
# ML Model - 1 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)
lasso = Lasso()
parameters = {'alpha': [1e-15,1e-13,1e-10,1e-8,1e-5,1e-4,1e-3,.011,1e-2,.008,1e-1,1,5,10,20,30,40,45,50,55,60,100]}
# Fit the Algorithm
lasso_regressor = GridSearchCV(lasso, parameters, scoring='neg_mean_squared_error', cv=4)
lasso_regressor.fit(X_train, y_train)


In [ ]:
print("The best fit alpha value is found out to be :" ,lasso_regressor.best_params_)
print("\nUsing ",lasso_regressor.best_params_, " the negative mean squared error is: ", lasso_regressor.best_score_)

In [ ]:
# Predict on the model
y_pred_lasso = lasso_regressor.predict(X_test)

##Evaluation metrics

In [ ]:
eval_metric(y_test,y_pred_lasso)

In [ ]:
plt.figure(figsize=(15,10))
plt.plot((y_pred_lasso)**2)
plt.plot(np.array((y_test))**2)
plt.legend(["Predicted","Actual"])
plt.show()

In [ ]:
#Resuldual Analysis
fig=plt.figure(figsize=(8,8))

sns.distplot(((y_test)**2- (y_pred_lasso)**2),bins=20,color='r')

#Plot Label
fig.suptitle('Residual Analysis', fontsize = 20)

In [ ]:
### Heteroscadacity
plt.scatter((y_pred_lasso)**2,(y_test)**2-(y_pred_lasso)**2,c='r')
plt.xlabel('Predicted selling price')
plt.ylabel('residuals')

###Hyperperameter tuning and cross validation with Ridge

In [ ]:
# Hyperprarameter tuning with Ridge
ridge = Ridge()
parameters = {'alpha': [1e-15,1e-10,1e-8,1e-5,1e-4,1e-3,1e-2,1,5,9,10,11,20,30,40,45,50,55,60,100,9]}
ridge_regressor = GridSearchCV(ridge, parameters, scoring='neg_mean_squared_error', cv=3)
ridge_regressor.fit(X_train,y_train)

In [ ]:
print("The best fit alpha value is found out to be :" ,ridge_regressor.best_params_)
print("\nUsing ",ridge_regressor.best_params_, " the negative mean squared error is: ", ridge_regressor.best_score_)

In [ ]:
y_pred_ridge = ridge_regressor.predict(X_test)

###Evaluation metrics

In [ ]:
eval_metric(y_test,y_pred_ridge)

In [ ]:
plt.figure(figsize=(15,10))
plt.plot((y_pred_ridge)**2)
plt.plot((np.array(y_test))**2)
plt.legend(["Predicted","Actual"])
plt.show()

In [ ]:
plt.scatter((y_test)**2, (y_pred_ridge)**2)
plt.xlabel('Actual ')
plt.ylabel('Predicted ')

In [ ]:
#Resuldual Analysis
fig=plt.figure(figsize=(8,8))

sns.distplot(((y_test)**2- (y_pred_ridge)**2),bins=20,color='r')

#Plot Label
fig.suptitle('Residual Analysis', fontsize = 20)

In [ ]:
### Heteroscadacity
plt.scatter((y_pred_ridge)**2,(y_test)**2-(y_pred_ridge)**2,c='r')
plt.xlabel('Predicted selling price')
plt.ylabel('residuals')

# **Linear regression model from statsmodels.api.**

In [ ]:
X = sm.add_constant(X) ## let's add an intercept (beta_0) to our model
model = sm.OLS(y, X).fit() ## sm.OLS(output, input)
predictions = model.predict(X)

In [ ]:
# Print out the statistics
model.summary()

If we see above result 'wind_speed', "day" has P>|t| is greater than 0.05 then we can ignore this one from our independen variable.....

# **Conclusion**

From above model we can conclude that below point:

* We have positive correlation with Hour . As day rises demand of bike increases till midnight and have minimum at morning 5 am.
* Temp, Wind_speed, Visibility,Solar_rad, have a positive correlation . We get more demand with increase in temp , visibility.
* Rain, Snow have negative correlation with demand of Bike .
* Where as Categorical column have Holiday -Have less demnand
and No demand on No_Funct_day.

Thanks:)